In [33]:
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import pandas as pd


# driver = webdriver.Chrome()
driver = webdriver.Edge()

In [ ]:
# driver = webdriver.Chrome()
driver = webdriver.Edge()

driver.get("https://fundresearch.fidelity.com/fund-screener/results/table/overview/averageAnnualReturnsYear3/desc/1?assetClass=&category=&order=assetClass%2Ccategory")
time.sleep(3)

In [50]:
data = []

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table", id="scrollable-results-table")

    for tr in table.find("tbody", id="results-tbody").find_all("tr"):
        td = tr.find_all("td") 
        if len(td) > 2:
            match = re.match(r"^(.*?)\s*\((.*?)\)$", td[4].text.strip())
            name = match.group(1)
            symbol = match.group(2)
            gross_er = td[13].text.strip()
            data.append({"symbol": symbol, "fund_name": name, "expense_ratio": gross_er})

    # Find all control items
    pagination = driver.find_element(By.CLASS_NAME, "pagination-controls")
    control_items = pagination.find_elements(By.CLASS_NAME, "control-item")


    # Loop through items to find the "Next" button
    for item in control_items:
        if item.text.strip().lower() == "next" and "disabled" not in item.get_attribute("class"):
            item.click()
            time.sleep(3)  # Wait for the next page to load
            break
    else:
        print("Next button is disabled or not found.")
        break


Next button is disabled or not found.


In [51]:
data

[{'symbol': 'BTCFX',
  'fund_name': 'Bitcoin ProFund Investor Class',
  'expense_ratio': '1.31%'},
 {'symbol': 'BTCLX',
  'fund_name': 'Vest Bitcoin Strategy Managed Volatility Fund Investor Class Shares',
  'expense_ratio': '3.44%'},
 {'symbol': 'BTCVX',
  'fund_name': '$Vest Bitcoin Strategy Managed Volatility Fund Institutional Class Shares',
  'expense_ratio': '2.96%'},
 {'symbol': 'EKWAX',
  'fund_name': 'Allspring Precious Metals Fund - Class A',
  'expense_ratio': '1.17%'},
 {'symbol': 'EKWCX',
  'fund_name': '$Allspring Precious Metals Fund - Class C',
  'expense_ratio': '1.92%'},
 {'symbol': 'FSELX',
  'fund_name': 'Fidelity® Select Semiconductors Portfolio',
  'expense_ratio': '0.62%'},
 {'symbol': 'OCMAX',
  'fund_name': 'OCM Gold Fund Atlas Class',
  'expense_ratio': '1.88%'},
 {'symbol': 'INIYX',
  'fund_name': '$VanEck International Investors Gold Fund Class Y',
  'expense_ratio': '1.13%'},
 {'symbol': 'OCMGX',
  'fund_name': '$OCM Gold Fund Investor Class',
  'expense_ra

In [52]:
driver.quit()

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.to_parquet("fidelity_info.parquet", index=False)

In [54]:
df.head()

,symbol,fund_name,expense_ratio
0,BTCFX,Bitcoin ProFund Investor Class,1.31%
1,BTCLX,Vest Bitcoin Strategy Managed Volatility Fund ...,3.44%
2,BTCVX,$Vest Bitcoin Strategy Managed Volatility Fund...,2.96%
3,EKWAX,Allspring Precious Metals Fund - Class A,1.17%
4,EKWCX,$Allspring Precious Metals Fund - Class C,1.92%
